In [1]:
from __future__ import print_function

import pandas as pd
import os
from dotenv import load_dotenv
from datetime import date, datetime, timedelta
import sqlalchemy
import pymysql

In [3]:
load_dotenv()

(host, user, password, port, database) = (os.getenv('host'), os.getenv('user'), os.getenv('passwd'), os.getenv('port'), os.getenv('database'))

In [102]:
def get_connection():
    return sqlalchemy.create_engine(
        url="mysql+pymysql://{0}:{1}@{2}:{3}/{4}".format(
            user, password, host, port, database
        )
    )

In [103]:
engine = get_connection()

table = "cpt_codes"

In [104]:
show_table_query = "DESCRIBE " + table
with engine.connect() as conn:
    result = conn.execute(show_table_query)
for row in result:
    print(row)

('key', 'int', 'NO', 'PRI', None, 'auto_increment')
('id', 'varchar(64)', 'NO', '', None, '')
('code', 'varchar(16)', 'NO', 'UNI', None, '')
('type', 'varchar(32)', 'NO', '', None, '')
('proprietary_name', 'varchar(256)', 'NO', '', None, '')
('proprietary_name_suffix', 'varchar(128)', 'YES', '', None, '')
('nonproprietary_name', 'varchar(512)', 'YES', '', None, '')
('dosage_form', 'varchar(64)', 'YES', '', None, '')
('route', 'varchar(128)', 'YES', '', None, '')
('start_marketing_date', 'date', 'YES', '', None, '')
('end_marketing_date', 'date', 'YES', '', None, '')
('marketing_category_name', 'varchar(45)', 'YES', '', None, '')
('application_number', 'varchar(20)', 'YES', '', None, '')
('labeler_name', 'varchar(128)', 'YES', '', None, '')
('substance_name', 'varchar(3450)', 'YES', '', None, '')
('active_numenator_strength', 'varchar(510)', 'YES', '', None, '')
('active_ingredient_unit', 'varchar(1500)', 'YES', '', None, '')
('pharma_class', 'varchar(4010)', 'YES', '', None, '')
('desc

In [105]:
product_path = 'ndcxls/PPRRVU23_JAN.xlsx'
start_row = 1
table_ndc = 'ndc_product'
columns = ['id', 'code', 'type', 'proprietary_name', 'proprietary_name_suffix', 'nonproprietary_name', 'dosage_form', 'route', 'start_marketing_date', 'end_marketing_date', 
    'marketing_category_name', 'application_number', 'labeler_name', 'substance_name', 'active_numenator_strength', 'active_ingredient_unit', 'pharma_class', 
    'descheduled','ndc_exclude', 'listing_cerified_through']
sqltypes_ndc = {'id': sqlalchemy.types.VARCHAR(length=64), 'code': sqlalchemy.types.VARCHAR(length=16), 'type': sqlalchemy.types.VARCHAR(length=32), 
    'proprietary_name': sqlalchemy.types.VARCHAR(length=256), 'proprietary_name_suffix': sqlalchemy.types.VARCHAR(length=128), 'nonproprietary_name': sqlalchemy.types.VARCHAR(length=512), 
    'dosage_form': sqlalchemy.types.VARCHAR(length=64), 'route': sqlalchemy.types.VARCHAR(length=128), 'start_marketing_date': sqlalchemy.DateTime() , 
    'end_marketing_date': sqlalchemy.DateTime(), 'marketing_category_name': sqlalchemy.types.VARCHAR(length=45), 'application_number': sqlalchemy.types.VARCHAR(length=20), 
    'labeler_name': sqlalchemy.types.VARCHAR(length=128), 'substance_name': sqlalchemy.types.VARCHAR(length=3450), 'active_numenator_strength': sqlalchemy.types.VARCHAR(length=510), 
    'active_ingredient_unit': sqlalchemy.types.VARCHAR(length=1500), 'pharma_class': sqlalchemy.types.VARCHAR(length=4010), 'descheduled':sqlalchemy.types.VARCHAR(length=12),
    'ndc_exclude':sqlalchemy.types.VARCHAR(length=20), 'listing_cerified_through': sqlalchemy.DateTime()}

df = pd.read_excel(product_path, header=None, names=columns, skiprows= start_row)
df.columns = columns

In [109]:
df['start_marketing_date'] =  pd.to_datetime(df['start_marketing_date'], format='%Y%m%d')
df['end_marketing_date'] =  pd.to_datetime(df['end_marketing_date'], format='%Y%m%d')
df['listing_cerified_through'] =  pd.to_datetime(df['listing_cerified_through'], format='%Y%m%d')
df.drop_duplicates(subset = ['code'], inplace=True)
df.head()

,id,code,type,proprietary_name,proprietary_name_suffix,nonproprietary_name,dosage_form,route,start_marketing_date,end_marketing_date,marketing_category_name,application_number,labeler_name,substance_name,active_numenator_strength,active_ingredient_unit,pharma_class,descheduled,ndc_exclude,listing_cerified_through
0,0002-0800_b02ed630-6947-431a-a8c8-227571403941,0002-0800,HUMAN OTC DRUG,Sterile Diluent,NaN,diluent,"INJECTION, SOLUTION",SUBCUTANEOUS,1987-07-10,NaT,BLA,BLA018781,Eli Lilly and Company,WATER,1,mL/mL,NaN,NaN,N,2023-12-31
1,0002-1200_c731b9c5-eaa5-4310-9c0a-11df9268b9ae,0002-1200,HUMAN PRESCRIPTION DRUG,Amyvid,NaN,Florbetapir F 18,"INJECTION, SOLUTION",INTRAVENOUS,2012-06-01,NaT,NDA,NDA202008,Eli Lilly and Company,FLORBETAPIR F-18,51,mCi/mL,"Positron Emitting Activity [MoA], Radioactive ...",NaN,N,2023-12-31
2,0002-1210_d03b2693-0231-4df4-a037-63017a42e85a,0002-1210,HUMAN PRESCRIPTION DRUG,TAUVID,NaN,Flortaucipir F-18,"INJECTION, SOLUTION",INTRAVENOUS,2020-05-28,NaT,NDA,NDA212123,Eli Lilly and Company,FLORTAUCIPIR F-18,51,mCi/mL,NaN,NaN,N,2023-12-31
3,0002-1220_d03b2693-0231-4df4-a037-63017a42e85a,0002-1220,HUMAN PRESCRIPTION DRUG,TAUVID,NaN,Flortaucipir F-18,"INJECTION, SOLUTION",INTRAVENOUS,2022-07-01,NaT,NDA,NDA212123,Eli Lilly and Company,FLORTAUCIPIR F-18,100,mCi/mL,NaN,NaN,N,2023-12-31
4,0002-1433_547fdc66-8f4c-422d-ac7d-b78865738ce0,0002-1433,HUMAN PRESCRIPTION DRUG,Trulicity,NaN,Dulaglutide,"INJECTION, SOLUTION",SUBCUTANEOUS,2014-09-18,NaT,BLA,BLA125469,Eli Lilly and Company,DULAGLUTIDE,0.75,mg/.5mL,"GLP-1 Receptor Agonist [EPC], Glucagon-Like Pe...",NaN,N,2023-12-31


In [107]:
df.dtypes

id                                   object
code                                 object
type                                 object
proprietary_name                     object
proprietary_name_suffix              object
nonproprietary_name                  object
dosage_form                          object
route                                object
start_marketing_date         datetime64[ns]
end_marketing_date           datetime64[ns]
marketing_category_name              object
application_number                   object
labeler_name                         object
substance_name                       object
active_numenator_strength            object
active_ingredient_unit               object
pharma_class                         object
descheduled                          object
ndc_exclude                          object
listing_cerified_through     datetime64[ns]
dtype: object

In [111]:
df.to_sql(name=table_ndc,con=engine, if_exists='append', index=False, chunksize=100, dtype=sqltypes_ndc)

96547